In [ ]:
import pandas as pd, numpy as np
def getRecBipart(cov, sortIx):
    # Compute HRP allocation
    w = pd.Series(1.0, index=sortIx)
    clusters = [sortIx]    
    while len(clusters) > 0:
        # Bisect each cluster
        new_clusters = []
        for cluster in clusters:
            if len(cluster) > 1:
                split = int(len(cluster) / 2)
                new_clusters.append(cluster[:split])
                new_clusters.append(cluster[split:])
        clusters = new_clusters
        # Update weights
        for i in range(0, len(clusters), 2):
            c1 = clusters[i]
            c2 = clusters[i + 1]
            var1 = getClusterVar(cov, c1)
            var2 = getClusterVar(cov, c2)
            alpha = 1.0 - var1 / (var1 + var2)
            w[c1] *= alpha
            w[c2] *= 1.0 - alpha
    return w
#------------------------------------------------------------------------------
def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar
#------------------------------------------------------------------------------
def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp